# Akıllı Sürücü İzleme Sistemi

Bu proje; sürücü simülasyonlarından üretilen oturum kayıtlarını (JSON) yönetebilen,
web tabanlı bir kontrol paneli sunan ve sürüş risk metriklerini raporlayan bir sistemdir.

Öne çıkan özellikler:
- Sürücü profili (ad, soyad, plaka) yönetimi
- Oturum geçmişi listeleme, oturum detayı görüntüleme
- Oturum karşılaştırma ekranı
- PDF rapor export (ReportLab ile)
- Telegram bot üzerinden acil durum kişilerine bildirim gönderme
- Basit dashboard kartları (toplam oturum, toplam süre vb.)

Notebook içinde çalıştırırken:
- Flask sunucusu hücresini en sonda çalıştırın.



In [1]:
#Standart kütüphaneler 
import os
import json
import shutil
from datetime import datetime
from collections import defaultdict
from io import BytesIO

#Flask
from flask import (
    Flask,
    request,
    redirect,
    url_for,
    render_template_string,
    send_from_directory,
    send_file,
    jsonify,
)

#Projenin ana sürücü izleme sistemi
from driver_system import FullDriverMonitoringSystem


TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN", "").strip()
TELEGRAM_CHAT_ID = os.getenv("TELEGRAM_CHAT_ID", "").strip()

pygame 2.6.1 (SDL 2.28.4, Python 3.10.19)
Hello from the pygame community. https://www.pygame.org/contribute.html
[OK] Ses sistemi (pygame) baslatildi


C:\Users\gts_-\anaconda3\envs\gorsel\lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [2]:
# PDF TR -> ASCII dönüştürücü

TR_ASCII_MAP = str.maketrans({
    "ç": "c", "Ç": "C",
    "ğ": "g", "Ğ": "G",
    "ı": "i", "İ": "I",
    "ö": "o", "Ö": "O",
    "ş": "s", "Ş": "S",
    "ü": "u", "Ü": "U",
})

def to_ascii_tr(text):
    if text is None:
        return ""
    if not isinstance(text, str):
        text = str(text)
    return text.translate(TR_ASCII_MAP)


In [3]:
# Notebook içinde __file__ bazen yoktur; o yüzden try/except ile güvenli alıyoruz

try:
    BASE_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    BASE_DIR = os.getcwd()

SESSIONS_DIR = os.path.join(BASE_DIR, "sessions")
os.makedirs(SESSIONS_DIR, exist_ok=True)

PROFILE_FILE = os.path.join(BASE_DIR, "driver_profile_info.json")
EMERGENCY_CONTACTS_FILE = os.path.join(BASE_DIR, "emergency_contacts.json")


In [4]:
# PDF rapor özelliği için kontrolü

PDF_AVAILABLE = False
FONT_NAME = "Helvetica"

try:
    from reportlab.lib.pagesizes import A4
    from reportlab.lib import colors
    from reportlab.lib.units import inch
    from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
    from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
    from reportlab.lib.enums import TA_CENTER
    from reportlab.pdfbase import pdfmetrics
    from reportlab.pdfbase.ttfonts import TTFont

    FONT_PATH = os.path.join(BASE_DIR, "DejaVuSans.ttf")
    if os.path.exists(FONT_PATH):
        pdfmetrics.registerFont(TTFont("DejaVu", FONT_PATH))
        FONT_NAME = "DejaVu"

    PDF_AVAILABLE = True
except Exception:
    PDF_AVAILABLE = False
    print("[UYARI] reportlab yüklü değil. PDF export devre dışı.")


In [5]:
def load_profile_info():
    if os.path.exists(PROFILE_FILE):
        try:
            with open(PROFILE_FILE, "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception:
            pass
    return {"name": "", "surname": "", "plate": ""}


def save_profile_info(name, surname, plate):
    data = {"name": name.strip(), "surname": surname.strip(), "plate": plate.strip()}
    with open(PROFILE_FILE, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    return data


def reset_profile():
    if os.path.exists(PROFILE_FILE):
        try:
            os.remove(PROFILE_FILE)
        except Exception:
            pass


In [6]:
def safe_read_json(path: str):
    try:
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception:
        return None


def is_session_like(data: dict) -> bool:
    if not isinstance(data, dict):
        return False
    return ("statistics" in data) or ("records" in data) or ("start_time" in data and "end_time" in data)


def make_unique_session_filename(prefix="session", ext=".json"):
    return f"{prefix}_{datetime.now().strftime('%Y%m%d_%H%M%S_%f')}{ext}"


def ingest_root_sessions_to_folder():
    """
    BASE_DIR kökünde yanlışlıkla duran session json'larını sessions/ içine kopyalar.
    """
    moved = 0
    for fname in os.listdir(BASE_DIR):
        if not fname.lower().endswith(".json"):
            continue
        if fname in {"driver_profile_info.json", "driver_profile.json", "profile.json", "emergency_contacts.json"}:
            continue

        root_path = os.path.join(BASE_DIR, fname)
        if os.path.isdir(root_path):
            continue

        data = safe_read_json(root_path)
        if not data or not is_session_like(data):
            continue

        new_name = make_unique_session_filename(prefix="session")
        dst = os.path.join(SESSIONS_DIR, new_name)
        try:
            shutil.copy2(root_path, dst)
            moved += 1
        except Exception:
            pass

    return moved


In [7]:
def delete_all_sessions():
    os.makedirs(SESSIONS_DIR, exist_ok=True)
    removed = 0
    for fname in os.listdir(SESSIONS_DIR):
        if fname.endswith(".json"):
            try:
                os.remove(os.path.join(SESSIONS_DIR, fname))
                removed += 1
            except Exception:
                pass
    return removed


def list_sessions():
    os.makedirs(SESSIONS_DIR, exist_ok=True)
    files = []
    for fname in os.listdir(SESSIONS_DIR):
        if not fname.endswith(".json"):
            continue

        path = os.path.join(SESSIONS_DIR, fname)
        data = safe_read_json(path)
        if not data:
            continue

        start_time = data.get("start_time", "")
        sort_key = None
        try:
            if start_time:
                sort_key = datetime.fromisoformat(start_time[:26])
        except Exception:
            sort_key = None

        if sort_key is None:
            try:
                sort_key = datetime.fromtimestamp(os.path.getmtime(path))
            except Exception:
                sort_key = datetime.min

        files.append(
            {
                "filename": fname,
                "start_time": data.get("start_time", ""),
                "end_time": data.get("end_time", ""),
                "stats": data.get("statistics", {}) or {},
                "_sort": sort_key,
            }
        )

    files.sort(key=lambda x: x["_sort"], reverse=True)
    for x in files:
        x.pop("_sort", None)
    return files


def get_session_detail(filename):
    path = os.path.join(SESSIONS_DIR, filename)
    if not os.path.exists(path):
        return None
    return safe_read_json(path)


In [8]:
def calculate_dashboard_stats():
    """
    Üst kartlar için:
      - total_sessions
      - total_minutes
      - avg_drowsy
      - avg_distracted
    """
    sessions = list_sessions()
    if not sessions:
        return {
            "total_sessions": 0,
            "total_duration_seconds": 0,
            "total_minutes": 0,
            "avg_drowsy": 0,
            "avg_distracted": 0,
            "total_blinks": 0,
            "emotion_distribution": {"normal": 0, "tired": 0},
            "recent_trend": [],
        }

    total_duration = 0
    drowsy_sum = 0
    distracted_sum = 0
    total_blinks = 0
    emotion_counts = defaultdict(float)
    recent_trend = []

    for session in sessions:
        stats = session.get("stats", {}) or {}
        total_duration += float(stats.get("total_duration_seconds", 0) or 0)
        drowsy_sum += float(stats.get("drowsy_percentage", 0) or 0)
        distracted_sum += float(stats.get("distracted_percentage", 0) or 0)
        total_blinks += int(stats.get("total_blinks", 0) or 0)

        emotion_dist = stats.get("emotion_distribution", {}) or {}
        for emotion, percentage in emotion_dist.items():
            try:
                emotion_counts[emotion] += float(percentage or 0)
            except Exception:
                pass

        if len(recent_trend) < 7:
            recent_trend.append(
                {
                    "date": session.get("start_time", "")[:10] if session.get("start_time") else "",
                    "drowsy": float(stats.get("drowsy_percentage", 0) or 0),
                    "distracted": float(stats.get("distracted_percentage", 0) or 0),
                }
            )

    num_sessions = len(sessions)
    total_minutes = int(round(total_duration / 60.0)) if total_duration else 0

    return {
        "total_sessions": num_sessions,
        "total_duration_seconds": total_duration,
        "total_minutes": total_minutes,
        "avg_drowsy": (drowsy_sum / num_sessions) if num_sessions else 0,
        "avg_distracted": (distracted_sum / num_sessions) if num_sessions else 0,
        "total_blinks": total_blinks,
        "emotion_distribution": {
            "normal": (emotion_counts.get("normal", 0) / num_sessions) if num_sessions else 0,
            "tired": (emotion_counts.get("tired", 0) / num_sessions) if num_sessions else 0,
        },
        "recent_trend": list(reversed(recent_trend)),
    }


def calculate_risk_score(stats):
    """
    Risk hesabı: drowsy + distracted + tired ağırlıklı.
    """
    drowsy = float(stats.get("drowsy_percentage", 0) or 0)
    distracted = float(stats.get("distracted_percentage", 0) or 0)
    emotion_dist = stats.get("emotion_distribution", {}) or {}
    tired = float(emotion_dist.get("tired", 0) or 0)

    risk = (drowsy * 0.45) + (distracted * 0.45) + (tired * 0.10)
    return min(100, max(0, risk))


In [9]:
def generate_pdf_report(session_data, filename):
    # TR -> ASCII dönüştürücü
    p = to_ascii_tr

    if not PDF_AVAILABLE:
        return None

    buffer = BytesIO()
    doc = SimpleDocTemplate(
        buffer,
        pagesize=A4,
        topMargin=0.5 * inch,
        bottomMargin=0.5 * inch
    )
    story = []
    styles = getSampleStyleSheet()

    title_style = ParagraphStyle(
        "CustomTitle",
        parent=styles["Heading1"],
        fontName=FONT_NAME,
        fontSize=24,
        textColor=colors.HexColor("#3b82f6"),
        spaceAfter=30,
        alignment=TA_CENTER,
    )

    normal_style = ParagraphStyle(
        "NormalTR",
        parent=styles["Normal"],
        fontName=FONT_NAME,
        fontSize=11,
        textColor=colors.black,
    )

    heading2 = ParagraphStyle(
        "Heading2TR",
        parent=styles["Heading2"],
        fontName=FONT_NAME,
        fontSize=14,
        textColor=colors.HexColor("#111827"),
    )

    footer_style = ParagraphStyle(
        "Footer",
        parent=styles["Normal"],
        fontName=FONT_NAME,
        fontSize=8,
        textColor=colors.grey,
        alignment=TA_CENTER,
    )

    def table_ascii(data):
        out = []
        for row in data:
            out.append([p(cell) for cell in row])
        return out

    stats = session_data.get("statistics", {}) or {}
    profile = load_profile_info()
    risk_score = calculate_risk_score(stats)

    story.append(Paragraph(p("Surucu Izleme Raporu"), title_style))
    story.append(Spacer(1, 0.2 * inch))

    info_data = table_ascii([
        ["Surucu Adi", f"{profile.get('name', '')} {profile.get('surname', '')}"],
        ["Plaka", profile.get("plate", "N/A")],
        ["Dosya", filename],
        ["Baslangic", session_data.get("start_time", "N/A")[:19] if session_data.get("start_time") else "N/A"],
        ["Bitis", session_data.get("end_time", "N/A")[:19] if session_data.get("end_time") else "N/A"],
        ["Sure", f"{stats.get('total_duration_seconds', 0)} saniye"],
    ])

    info_table = Table(info_data, colWidths=[2.0 * inch, 4.0 * inch])
    info_table.setStyle(TableStyle([
        ("FONTNAME", (0, 0), (-1, -1), FONT_NAME),
        ("BACKGROUND", (0, 0), (0, -1), colors.HexColor("#1e293b")),
        ("TEXTCOLOR", (0, 0), (-1, -1), colors.white),
        ("ALIGN", (0, 0), (-1, -1), "LEFT"),
        ("FONTSIZE", (0, 0), (-1, -1), 10),
        ("BOTTOMPADDING", (0, 0), (-1, -1), 10),
        ("BACKGROUND", (1, 0), (1, -1), colors.HexColor("#0f172a")),
        ("GRID", (0, 0), (-1, -1), 1, colors.HexColor("#334155")),
    ]))
    story.append(info_table)
    story.append(Spacer(1, 0.3 * inch))

    story.append(Paragraph(p("Istatistikler"), heading2))
    story.append(Spacer(1, 0.1 * inch))

    stats_data = table_ascii([
        ["Metrik", "Deger"],
        ["Toplam Sure", f"{stats.get('total_duration_seconds', 0)} saniye"],
        ["Ortalama Goz Acikligi", f"{float(stats.get('average_eye_openness', 0) or 0):.2f}"],
        ["Uykululuk Orani", f"{float(stats.get('drowsy_percentage', 0) or 0):.1f}%"],
        ["Dikkat Dag. Orani", f"{float(stats.get('distracted_percentage', 0) or 0):.1f}%"],
        ["Risk Skoru", f"{risk_score:.0f}/100"],
    ])

    stats_table = Table(stats_data, colWidths=[3 * inch, 3 * inch])
    stats_table.setStyle(TableStyle([
        ("FONTNAME", (0, 0), (-1, -1), FONT_NAME),
        ("BACKGROUND", (0, 0), (-1, 0), colors.HexColor("#3b82f6")),
        ("TEXTCOLOR", (0, 0), (-1, 0), colors.white),
        ("FONTSIZE", (0, 0), (-1, 0), 12),
        ("BOTTOMPADDING", (0, 0), (-1, 0), 10),
        ("BACKGROUND", (0, 1), (-1, -1), colors.HexColor("#111827")),
        ("TEXTCOLOR", (0, 1), (-1, -1), colors.white),
        ("GRID", (0, 0), (-1, -1), 1, colors.HexColor("#334155")),
        ("ROWBACKGROUNDS", (0, 1), (-1, -1), [colors.HexColor("#1f2937"), colors.HexColor("#111827")]),
    ]))
    story.append(stats_table)
    story.append(Spacer(1, 0.3 * inch))

    emotion_dist = stats.get("emotion_distribution", {}) or {}
    story.append(Paragraph(p("Duygu Dagilimi"), heading2))
    story.append(Spacer(1, 0.1 * inch))

    emotion_data = table_ascii([
        ["Duygu", "Yuzde"],
        ["Normal", f"{float(emotion_dist.get('normal', 0) or 0):.1f}%"],
        ["Yorgun", f"{float(emotion_dist.get('tired', 0) or 0):.1f}%"],
    ])

    emotion_table = Table(emotion_data, colWidths=[3 * inch, 3 * inch])
    emotion_table.setStyle(TableStyle([
        ("FONTNAME", (0, 0), (-1, -1), FONT_NAME),
        ("BACKGROUND", (0, 0), (-1, 0), colors.HexColor("#10b981")),
        ("TEXTCOLOR", (0, 0), (-1, 0), colors.white),
        ("FONTSIZE", (0, 0), (-1, 0), 12),
        ("BOTTOMPADDING", (0, 0), (-1, 0), 10),
        ("BACKGROUND", (0, 1), (-1, -1), colors.HexColor("#111827")),
        ("TEXTCOLOR", (0, 1), (-1, -1), colors.white),
        ("GRID", (0, 0), (-1, -1), 1, colors.HexColor("#334155")),
    ]))
    story.append(emotion_table)
    story.append(Spacer(1, 0.25 * inch))

    story.append(Paragraph(p("Oneriler"), heading2))
    story.append(Spacer(1, 0.1 * inch))

    recommendations = []
    if risk_score >= 60:
        recommendations.append("Yuksek risk tespit edildi. Daha sik mola verin ve yeterince dinlenin.")
    if float(stats.get("drowsy_percentage", 0) or 0) > 20:
        recommendations.append("Uykululuk orani yuksek. Uyku duzeninize dikkat edin.")
    if float(stats.get("distracted_percentage", 0) or 0) > 15:
        recommendations.append("Dikkat daginikligi tespit edildi. Surus sirasinda dikkatinizi toplayin.")
    if not recommendations:
        recommendations.append("Genel olarak guvenli bir surus gerceklestirdiniz. Tebrikler!")

    for rec in recommendations:
        story.append(Paragraph(p(f"• {rec}"), normal_style))
        story.append(Spacer(1, 0.06 * inch))

    story.append(Spacer(1, 0.25 * inch))
    story.append(Paragraph(
        p(f"Rapor Olusturulma Tarihi: {datetime.now().strftime('%d.%m.%Y %H:%M')}"),
        footer_style
    ))

    doc.build(story)
    buffer.seek(0)
    return buffer


In [10]:
def get_bot_info():
    token = os.getenv("TELEGRAM_BOT_TOKEN", "").strip()
    if not token:
        return None, "TELEGRAM_BOT_TOKEN tanımlı değil"

    try:
        import requests
    except Exception:
        return None, "requests kütüphanesi yüklü değil (pip install requests)"

    url = f"https://api.telegram.org/bot{token}/getMe"
    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            data = r.json()
            if data.get("ok"):
                return data.get("result", {}), None
        elif r.status_code == 401:
            return None, "Bot token geçersiz (401 Unauthorized)"
        return None, f"Bot bilgisi alınamadı: HTTP {r.status_code}"
    except Exception as e:
        return None, f"Bağlantı hatası: {str(e)}"


def send_telegram_message(chat_id: str, text: str):
    token = os.getenv("TELEGRAM_BOT_TOKEN", "").strip()
    if not token:
        return False, "❌ TELEGRAM_BOT_TOKEN tanımlı değil"

    try:
        import requests
    except Exception:
        return False, "❌ requests kütüphanesi yüklü değil (pip install requests)"

    url = f"https://api.telegram.org/bot{token}/sendMessage"
    payload = {"chat_id": str(chat_id).strip(), "text": text, "parse_mode": "HTML"}

    try:
        r = requests.post(url, json=payload, timeout=15)
        if r.status_code == 200:
            jd = r.json()
            if jd.get("ok"):
                return True, "✅ Mesaj gönderildi"
            return False, f"API hatası: {jd.get('description', 'Bilinmeyen hata')}"
        if r.status_code == 401:
            return False, "❌ Bot token geçersiz (401 Unauthorized)"
        if r.status_code == 403:
            return False, "❌ Bot engellenmiş veya chat erişimi yok (403 Forbidden)"
        if r.status_code == 400:
            return False, "❌ Chat bulunamadı veya bot erişemiyor (400 Bad Request)"
        return False, f"❌ HTTP {r.status_code}: {r.text[:200]}"
    except Exception as e:
        return False, f"❌ Bağlantı hatası: {str(e)}"


def test_telegram_connection():
    bot_info, error = get_bot_info()
    if error:
        return {"ok": False, "error": error}
    return {
        "ok": True,
        "bot_username": bot_info.get("username"),
        "bot_name": bot_info.get("first_name"),
        "bot_id": bot_info.get("id"),
    }


In [11]:
def load_emergency_contacts():
    if os.path.exists(EMERGENCY_CONTACTS_FILE):
        data = safe_read_json(EMERGENCY_CONTACTS_FILE)
        if isinstance(data, list):
            normalized = []
            for x in data:
                if not isinstance(x, dict):
                    continue
                normalized.append(x)
            return normalized
    return []


def save_emergency_contacts(items: list):
    try:
        with open(EMERGENCY_CONTACTS_FILE, "w", encoding="utf-8") as f:
            json.dump(items, f, ensure_ascii=False, indent=2)
    except Exception:
        pass
    return items


def add_emergency_contact(name: str, chat_id: str):
    items = load_emergency_contacts()
    new_item = {
        "id": f"c_{datetime.now().strftime('%Y%m%d_%H%M%S_%f')}",
        "name": (name or "").strip(),
        "chat_id": (chat_id or "").strip(),
        "created_at": datetime.now().isoformat(timespec="seconds"),
    }
    if not new_item["name"] or not new_item["chat_id"]:
        return None, "İsim ve Chat ID zorunlu."
    items.insert(0, new_item)
    save_emergency_contacts(items)
    return new_item, None


def delete_emergency_contact(contact_id: str):
    items = load_emergency_contacts()
    before = len(items)
    items = [x for x in items if x.get("id") != contact_id]
    save_emergency_contacts(items)
    return before - len(items)


def build_emergency_message(reason: str, seconds: float = None, extra: dict = None):
    profile = load_profile_info()
    extra = extra or {}
    now_str = datetime.now().strftime("%d.%m.%Y %H:%M:%S")

    sec_text = ""
    try:
        if seconds is not None:
            sec_text = f"\n⏱️ Süre: {float(seconds):.0f} saniye"
    except Exception:
        sec_text = ""

    location_text = ""
    if extra.get("lat") and extra.get("lon"):
        location_text = f"\n📍 Konum: {extra.get('lat')}, {extra.get('lon')}"

    session_file = extra.get("session_filename")
    session_text = f"\n📂 Oturum: {session_file}" if session_file else ""

    msg = (
        "🚨 <b>ACİL DURUM UYARISI</b>\n\n"
        f"👤 Sürücü: <b>{profile.get('name','')} {profile.get('surname','')}</b>\n"
        f"🚗 Plaka: <b>{profile.get('plate','')}</b>\n"
        f"⚠️ Durum: <b>{reason}</b>{sec_text}\n"
        f"🕐 Zaman: {now_str}"
        f"{location_text}"
        f"{session_text}\n\n"
        "❗ Lütfen sürücüyü kontrol edin."
    )
    return msg


In [12]:
app = Flask(__name__)
app.config["SECRET_KEY"] = "driver-monitor-secret"


In [13]:

PAGE_TEMPLATE = r"""
<!doctype html>
<html lang="tr">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Akıllı Sürücü İzleme Sistemi</title>
  <style>
    * { margin: 0; padding: 0; box-sizing: border-box; }

    :root {
      --primary: #3b82f6;
      --success: #10b981;
      --danger: #ef4444;
      --warning: #f59e0b;
      --bg-dark: #0f172a;
      --bg-card: rgba(30, 41, 59, 0.85);
      --bg-hover: rgba(51, 65, 85, 0.9);
      --text-primary: #f1f5f9;
      --text-secondary: #94a3b8;
      --text-muted: #64748b;
      --border: rgba(51, 65, 85, 0.7);
      --shadow: 0 10px 25px rgba(0, 0, 0, 0.35);
      --shadow-lg: 0 20px 40px rgba(0, 0, 0, 0.45);
      --gradient-primary: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    }

    [data-theme="light"] {
      --bg-dark: #f8fafc;
      --bg-card: rgba(255, 255, 255, 0.9);
      --bg-hover: rgba(241, 245, 249, 0.95);
      --text-primary: #0f172a;
      --text-secondary: #475569;
      --text-muted: #94a3b8;
      --border: rgba(226, 232, 240, 0.9);
      --shadow: 0 10px 25px rgba(0, 0, 0, 0.08);
      --shadow-lg: 0 20px 40px rgba(0, 0, 0, 0.12);
    }

    body {
      font-family: 'Inter', -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
      background: radial-gradient(1200px 600px at 50% -10%, rgba(59,130,246,.35), transparent 60%),
                  radial-gradient(900px 450px at 10% 0%, rgba(16,185,129,.25), transparent 55%),
                  radial-gradient(900px 450px at 90% 0%, rgba(245,158,11,.20), transparent 55%),
                  var(--bg-dark);
      color: var(--text-primary);
      line-height: 1.6;
      min-height: 100vh;
      padding: 20px;
      transition: background-color 0.3s ease, color 0.3s ease;
    }

    .container { max-width: 1400px; margin: 0 auto; padding: 0 20px; }

    .theme-toggle {
      position: fixed; top: 20px; right: 20px; z-index: 1000;
      background: var(--bg-card); border: 1px solid var(--border);
      border-radius: 999px; padding: 12px 18px; cursor: pointer;
      display: flex; align-items: center; gap: 10px; font-weight: 700;
      color: var(--text-primary); transition: all 0.25s ease;
      box-shadow: var(--shadow);
      backdrop-filter: blur(8px);
    }
    .theme-toggle:hover { transform: translateY(-2px); box-shadow: var(--shadow-lg); border-color: rgba(59,130,246,.7); background: var(--bg-hover); }

    .header { text-align: center; margin-bottom: 20px; padding: 10px 0; }
    .header h1 { font-size: 2.6rem; font-weight: 900; background: var(--gradient-primary); -webkit-background-clip: text; -webkit-text-fill-color: transparent; margin-bottom: 10px; }
    .header p { color: var(--text-secondary); font-size: 1.05rem; }

    /* --- ÜST KARTLAR --- */
    .stats-bar {
      display: grid;
      grid-template-columns: repeat(4, 1fr);
      gap: 18px;
      margin: 26px 0 26px;
    }
    @media (max-width: 1100px) { .stats-bar { grid-template-columns: repeat(2, 1fr); } }
    @media (max-width: 520px) { .stats-bar { grid-template-columns: 1fr; } }

    .stat-card {
      background: var(--bg-card);
      border: 1px solid var(--border);
      border-radius: 18px;
      padding: 22px 20px;
      box-shadow: var(--shadow);
      backdrop-filter: blur(10px);
      position: relative;
      overflow: hidden;
      min-height: 130px;
      display: flex;
      align-items: center;
      justify-content: center;
      text-align: center;
      transition: transform .25s ease, background .25s ease, border-color .25s ease;
    }
    .stat-card:hover { transform: translateY(-4px); background: var(--bg-hover); border-color: rgba(59,130,246,.55); }
    .stat-card::after{
      content:"";
      position:absolute; inset:-40px -40px auto auto;
      width:120px; height:120px;
      background: radial-gradient(circle at center, rgba(255,255,255,.10), transparent 70%);
      border-radius: 999px;
      transform: translate(20px,-20px);
      pointer-events:none;
    }
    .stat-icon { font-size: 2rem; margin-bottom: 10px; opacity:.95; }
    .stat-value { font-size: 2.2rem; font-weight: 900; margin: 4px 0 2px; letter-spacing: .5px; }
    .stat-label { font-size: .75rem; font-weight: 800; letter-spacing: 1.2px; color: var(--text-secondary); text-transform: uppercase; }

    .v-blue { color: #60a5fa; }
    .v-green { color: #34d399; }
    .v-red { color: #f87171; }
    .v-yellow { color: #fbbf24; }

    /* --- GRID --- */
    .grid { display: grid; grid-template-columns: 1fr 1fr; gap: 24px; margin-bottom: 30px; }
    @media (max-width: 1024px) { .grid { grid-template-columns: 1fr; } }

    .card {
      background: var(--bg-card);
      border-radius: 20px;
      padding: 26px;
      box-shadow: var(--shadow-lg);
      border: 1px solid var(--border);
      transition: all 0.25s ease;
      backdrop-filter: blur(10px);
    }
    .card:hover { transform: translateY(-3px); border-color: rgba(59,130,246,.5); background: var(--bg-hover); }

    .card-title { font-size: 1.45rem; font-weight: 900; margin-bottom: 22px; color: var(--text-primary); }
    .section-title { font-size: 1.05rem; font-weight: 800; margin-bottom: 14px; color: var(--text-primary); }

    .form-grid { display: grid; grid-template-columns: repeat(2, 1fr); gap: 16px; margin-bottom: 16px; }
    .form-group { display: flex; flex-direction: column; }
    .form-group.full-width { grid-column: 1 / -1; }

    label { font-size: 0.8rem; font-weight: 800; color: var(--text-secondary); margin-bottom: 8px; text-transform: uppercase; letter-spacing: .8px; }

    input[type="text"] {
      width: 100%;
      padding: 13px 14px;
      border-radius: 12px;
      border: 1px solid var(--border);
      background: rgba(15, 23, 42, 0.55);
      color: var(--text-primary);
      font-size: 1rem;
      transition: border-color 0.2s ease, box-shadow 0.2s ease;
    }
    [data-theme="light"] input[type="text"] { background: rgba(248, 250, 252, 0.9); }
    input[type="text"]:focus { outline: none; border-color: rgba(59,130,246,.8); box-shadow: 0 0 0 4px rgba(59,130,246,.14); }

    .btn {
      display: inline-flex; align-items: center; justify-content: center; gap: 8px;
      padding: 12px 18px; border-radius: 12px; border: none;
      font-size: 0.98rem; font-weight: 900; cursor: pointer;
      transition: transform .2s ease, filter .2s ease, background .2s ease, border-color .2s ease;
      text-decoration: none;
    }
    .btn:hover { transform: translateY(-1px); }
    .btn-primary { background: var(--gradient-primary); color: white; }
    .btn-secondary { background: transparent; color: var(--text-primary); border: 1px solid var(--border); }
    .btn-secondary:hover { background: rgba(255,255,255,.04); border-color: rgba(59,130,246,.6); }
    .btn-danger { background: rgba(239,68,68,.95); color: white; }
    .btn-danger:hover { filter: brightness(1.05); }

    .btn:disabled { opacity: 0.5; cursor: not-allowed; transform:none; }

    .btn-group { display: flex; gap: 10px; align-items: center; flex-wrap: wrap; }

    .badge { display: inline-flex; align-items: center; gap: 6px; padding: 6px 12px; border-radius: 999px; font-size: 0.72rem; font-weight: 900; letter-spacing: .8px; text-transform: uppercase; border: 1px solid var(--border); }
    .badge-ok { background: rgba(16,185,129,0.15); color: var(--success); }
    .badge-warn { background: rgba(239,68,68,0.15); color: var(--danger); }

    .message { padding: 14px 16px; border-radius: 12px; margin-top: 14px; font-size: 0.95rem; border: 1px solid var(--border); }
    .message-success { background: rgba(16, 185, 129, 0.12); color: var(--success); border-left: 4px solid var(--success); }
    .message-danger { background: rgba(239, 68, 68, 0.12); color: var(--danger); border-left: 4px solid var(--danger); }

    .divider { height: 1px; background: linear-gradient(90deg, transparent, var(--border), transparent); margin: 22px 0; border: none; }

    .table-container { overflow-x: auto; border-radius: 12px; border: 1px solid var(--border); }
    table { width: 100%; border-collapse: collapse; font-size: 0.86rem; }
    thead { background: rgba(15, 23, 42, 0.55); }
    th { padding: 12px 10px; text-align: left; font-weight: 900; color: var(--text-secondary); text-transform: uppercase; font-size: 0.7rem; letter-spacing: .9px; border-bottom: 1px solid var(--border); }
    td { padding: 12px 10px; border-bottom: 1px solid var(--border); color: var(--text-primary); }
    tbody tr:hover { background: rgba(255,255,255,.03); }

    .info-text { font-size: 0.86rem; color: var(--text-muted); margin-top: 10px; }

    .spinner { display: inline-block; width: 16px; height: 16px; border: 2px solid rgba(255,255,255,0.3); border-top-color: white; border-radius: 50%; animation: spin 0.8s linear infinite; }
    @keyframes spin { to { transform: rotate(360deg); } }

    .status-box { background: rgba(15, 23, 42, 0.45); padding: 14px; border-radius: 12px; border: 1px solid var(--border); }
  </style>
</head>
<body>
  <button class="theme-toggle" onclick="toggleTheme()">
    <span id="themeIcon">🌙</span>
    <span id="themeText">Dark Mode</span>
  </button>

  <div class="container">
    <div class="header">
      <h1>🚗 Akıllı Sürücü İzleme Sistemi</h1>
      <p>Profil bilgilerinizi doldurun, simülasyonu başlatın ve Telegram acil bildirimlerini yönetin</p>
    </div>

    <!-- ✅ ÜST KARTLAR (Senin görseldeki kısım) -->
    <div class="stats-bar">
      <div class="stat-card">
        <div>
          <div class="stat-icon">📊</div>
          <div class="stat-value v-blue">{{ dashboard_stats.total_sessions }}</div>
          <div class="stat-label">Toplam Oturum</div>
        </div>
      </div>

      <div class="stat-card">
        <div>
          <div class="stat-icon">⏱️</div>
          <div class="stat-value v-green">{{ dashboard_stats.total_minutes }}</div>
          <div class="stat-label">Toplam Dakika</div>
        </div>
      </div>

      <div class="stat-card">
        <div>
          <div class="stat-icon">😴</div>
          <div class="stat-value v-red">{{ "%.1f"|format(dashboard_stats.avg_drowsy or 0) }}%</div>
          <div class="stat-label">Ort. Uykululuk</div>
        </div>
      </div>

      <div class="stat-card">
        <div>
          <div class="stat-icon">👀</div>
          <div class="stat-value v-yellow">{{ "%.1f"|format(dashboard_stats.avg_distracted or 0) }}%</div>
          <div class="stat-label">Ort. Dikkat Dağ.</div>
        </div>
      </div>
    </div>

    <div class="grid">
      <!-- Sol Panel -->
      <div class="card">
        <h2 class="card-title">Sürücü Profili</h2>

        <form method="post" action="{{ url_for('save_profile') }}">
          <div class="form-grid">
            <div class="form-group">
              <label>Ad</label>
              <input type="text" name="name" value="{{ profile.name }}" required>
            </div>
            <div class="form-group">
              <label>Soyad</label>
              <input type="text" name="surname" value="{{ profile.surname }}" required>
            </div>
            <div class="form-group full-width">
              <label>Plaka</label>
              <input type="text" name="plate" value="{{ profile.plate }}" required>
            </div>
          </div>

          <div class="btn-group">
            <button class="btn btn-primary" type="submit">💾 Profili Kaydet</button>

            <!-- ✅ İSTEDİĞİN BUTONLAR -->
            <button class="btn btn-secondary" type="button" onclick="confirmResetProfile()">🧹 Profili Sil</button>
            <button class="btn btn-danger" type="button" onclick="confirmDeleteSessions()">🗑️ Oturumları Sil</button>

            {% if profile.name and profile.surname and profile.plate %}
              <span class="badge badge-ok">✓ Profil kayıtlı</span>
            {% else %}
              <span class="badge badge-warn">⚠ Profil eksik</span>
            {% endif %}
          </div>
        </form>

        <form id="resetProfileForm" method="post" action="{{ url_for('reset_profile_route') }}"></form>
        <form id="deleteSessionsForm" method="post" action="{{ url_for('delete_sessions_route') }}"></form>

        <hr class="divider">

        <h3 class="section-title">▶️ Simülasyon</h3>
        <form method="post" action="{{ url_for('start_drive') }}">
          <button class="btn btn-primary" type="submit"
                  {% if not profile.name or not profile.surname or not profile.plate %}disabled{% endif %}>
            Sürüşü Başlat
          </button>
        </form>

        {% if start_error %}
          <div class="message message-danger">{{ start_error }}</div>
        {% endif %}
        {% if start_success %}
          <div class="message message-success">{{ start_success }}</div>
        {% endif %}

        <hr class="divider">

        <h3 class="section-title">🔧 Telegram Durumu</h3>
        <div class="status-box" id="telegram-status">
          <div class="btn-group">
            <button class="btn btn-secondary" type="button" onclick="checkTelegramStatus()">
              🔄 Bağlantıyı Test Et
            </button>
            <button class="btn btn-secondary" type="button" onclick="getChatIdHelp()">
              📋 Chat ID Al
            </button>
          </div>
          <div class="info-text" style="margin-top:10px;">
            Chat ID almak için bot ile konuşmayı başlatıp <code>/start</code> yaz.
          </div>
        </div>

        <hr class="divider">

        <h3 class="section-title">🚨 Acil Durum Kişileri</h3>

        <form method="post" action="{{ url_for('add_contact') }}">
          <div class="form-grid">
            <div class="form-group">
              <label>İsim</label>
              <input type="text" name="contact_name" placeholder="Anne / Baba" required>
            </div>
            <div class="form-group">
              <label>Chat ID</label>
              <input type="text" name="contact_chat_id" placeholder="123456789" required>
            </div>
          </div>

          <div class="btn-group">
            <button class="btn btn-primary" type="submit">➕ Kişi Ekle</button>
            {% if contacts|length > 0 %}
              <span class="badge badge-ok">✓ {{ contacts|length }} kişi</span>
            {% else %}
              <span class="badge badge-warn">⚠ Kişi yok</span>
            {% endif %}
          </div>

          {% if contact_error %}
            <div class="message message-danger">{{ contact_error }}</div>
          {% endif %}
          {% if contact_success %}
            <div class="message message-success">{{ contact_success }}</div>
          {% endif %}
        </form>

        {% if contacts %}
          <div class="table-container" style="margin-top: 14px;">
            <table>
              <thead>
                <tr><th>İsim</th><th>Chat ID</th><th>İşlem</th></tr>
              </thead>
              <tbody>
                {% for c in contacts %}
                <tr>
                  <td><strong>{{ c.name }}</strong></td>
                  <td>{{ c.chat_id }}</td>
                  <td>
                    <button class="btn btn-secondary" style="padding: 8px 12px;" onclick="deleteContact('{{ c.id }}')">Sil</button>
                  </td>
                </tr>
                {% endfor %}
              </tbody>
            </table>
          </div>

          <div class="btn-group" style="margin-top: 12px;">
            <button class="btn btn-secondary" type="button" onclick="testEmergency()">🧪 Test Mesajı</button>
          </div>
        {% endif %}
      </div>

      <!-- Sağ Panel -->
      <div class="card">
        <h2 class="card-title">Sürüş Geçmişi</h2>

        {% if sessions %}
          <div class="table-container">
          <table>
            <thead>
              <tr>
                <th>Dosya</th>
                <th>Süre</th>
                <th>Uykululuk</th>
                <th>Dikkat</th>
              </tr>
            </thead>
            <tbody>
              {% for s in sessions %}
                <tr>
                  <td>
                    <a href="{{ url_for('session_detail', filename=s.filename) }}" style="color: var(--primary); text-decoration:none; font-weight:900;">
                      {{ s.filename }}
                    </a>
                    <a href="{{ url_for('download_session', filename=s.filename) }}" style="margin-left:10px; opacity:.7; text-decoration:none;">
                      ⬇️
                    </a>
                  </td>
                  <td>{{ s.stats.total_duration_seconds or 0 }} sn</td>
                  <td>{{ "%.1f"|format(s.stats.drowsy_percentage or 0) }}%</td>
                  <td>{{ "%.1f"|format(s.stats.distracted_percentage or 0) }}%</td>
                </tr>
              {% endfor %}
            </tbody>
          </table>
          </div>

          {% if sessions|length >= 2 %}
            <hr class="divider">
            <a class="btn btn-secondary" href="{{ url_for('compare_sessions') }}">📊 Oturum Karşılaştır</a>
          {% endif %}
        {% else %}
          <p style="text-align: center; padding: 40px; color: var(--text-muted);">Henüz kayıtlı oturum yok</p>
        {% endif %}
      </div>
    </div>
  </div>

  <script>
    function toggleTheme() {
      const html = document.documentElement;
      const current = html.getAttribute('data-theme');
      const newTheme = current === 'light' ? 'dark' : 'light';
      html.setAttribute('data-theme', newTheme);
      localStorage.setItem('theme', newTheme);
      document.getElementById('themeIcon').textContent = newTheme === 'light' ? '☀️' : '🌙';
      document.getElementById('themeText').textContent = newTheme === 'light' ? 'Light Mode' : 'Dark Mode';
    }
    (function() {
      const saved = localStorage.getItem('theme') || 'dark';
      document.documentElement.setAttribute('data-theme', saved);
      document.getElementById('themeIcon').textContent = saved === 'light' ? '☀️' : '🌙';
      document.getElementById('themeText').textContent = saved === 'light' ? 'Light Mode' : 'Dark Mode';
    })();

    function confirmResetProfile() {
      if (!confirm("Profil bilgilerini silmek istiyor musun?")) return;
      document.getElementById("resetProfileForm").submit();
    }

    function confirmDeleteSessions() {
      if (!confirm("TÜM oturum dosyaları silinecek. Emin misin?")) return;
      document.getElementById("deleteSessionsForm").submit();
    }

    async function checkTelegramStatus() {
      const statusDiv = document.getElementById('telegram-status');
      statusDiv.innerHTML = '<span class="spinner"></span> Kontrol ediliyor...';

      try {
        const r = await fetch('/api/telegram/test');
        const data = await r.json();

        if (data.ok) {
          statusDiv.innerHTML = `
            <div class="message message-success" style="margin:0;">
              ✅ Bot bağlantısı başarılı<br>
              <small>Bot: @${data.bot_username}</small>
            </div>
          `;
        } else {
          statusDiv.innerHTML = `<div class="message message-danger" style="margin:0;">${data.error}</div>`;
        }
      } catch (e) {
        statusDiv.innerHTML = `<div class="message message-danger" style="margin:0;">Bağlantı hatası</div>`;
      }
    }

    async function getChatIdHelp() {
      try {
        const r = await fetch('/api/telegram/get_chat_id');
        const data = await r.json();
        if (data.ok && data.getUpdates_url) {
          window.open(data.getUpdates_url, '_blank');
          alert('✅ Yeni sekmede URL açıldı\\n\\n"id": XXXXXX değerini kopyalayın');
        } else {
          alert('❌ ' + (data.error || 'Bot token tanımlı değil'));
        }
      } catch (e) {
        alert('❌ Hata: ' + e.message);
      }
    }

    async function deleteContact(id) {
      if (!confirm("Bu kişiyi silmek istiyor musun?")) return;
      const r = await fetch("/contacts/delete/" + encodeURIComponent(id), { method: "POST" });
      const j = await r.json();
      if (j.ok) window.location.reload();
      else alert("Silinemedi");
    }

    async function testEmergency() {
      if (!confirm("Tüm acil kişilere test mesajı gönderilecek. Emin misin?")) return;
      const r = await fetch("/api/emergency/trigger", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ reason: "TEST: Web panelinden deneme mesajı", seconds: 10 })
      });
      const j = await r.json();
      if (j.ok) {
        alert(`✅ Başarılı!\\n\\n${j.sent}/${j.total} kişiye mesaj gönderildi`);
      } else {
        alert(`❌ Hata: ${j.error}`);
      }
    }

    window.addEventListener('load', checkTelegramStatus);
  </script>
</body>
</html>
"""

# SESSION DETAIL + COMPARE (AŞAĞIYI DA SAKLIYORUZ)

SESSION_DETAIL_TEMPLATE = """
<!doctype html>
<html lang="tr">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Oturum Detayı - {{ session_data.filename }}</title>
  <script src="https://cdn.jsdelivr.net/npm/chart.js@4.4.0/dist/chart.umd.min.js"></script>
  <style>
    * { margin: 0; padding: 0; box-sizing: border-box; }
    :root {
      --primary: #3b82f6;
      --success: #10b981;
      --danger: #ef4444;
      --warning: #f59e0b;
      --bg-dark: #0f172a;
      --bg-card: #1e293b;
      --text-primary: #f1f5f9;
      --text-secondary: #94a3b8;
      --border: #334155;
    }
    [data-theme="light"] {
      --bg-dark: #f8fafc;
      --bg-card: #ffffff;
      --text-primary: #0f172a;
      --text-secondary: #475569;
      --border: #e2e8f0;
    }
    body {
      font-family: 'Inter', -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
      background: var(--bg-dark);
      color: var(--text-primary);
      line-height: 1.6;
      padding: 20px;
      transition: background-color 0.3s ease, color 0.3s ease;
    }
    .theme-toggle {
      position: fixed; top: 20px; right: 20px; z-index: 1000;
      background: var(--bg-card); border: 2px solid var(--border); border-radius: 50px;
      padding: 12px 24px; cursor: pointer; display: flex; align-items: center; gap: 10px;
      font-weight: 600; color: var(--text-primary); transition: all 0.3s ease;
    }
    .container { max-width: 1400px; margin: 0 auto; }
    .back-link { display: inline-flex; align-items: center; gap: 8px; color: var(--primary); text-decoration: none; margin-bottom: 20px; font-weight: 500; }
    .card { background: var(--bg-card); border-radius: 16px; padding: 24px; margin-bottom: 24px; border: 1px solid var(--border); }
    .card-title { font-size: 1.25rem; font-weight: 700; margin-bottom: 20px; }
    .stats-grid { display: grid; grid-template-columns: repeat(auto-fit, minmax(150px, 1fr)); gap: 16px; margin-bottom: 24px; }
    .stat-item { text-align: center; padding: 20px; background: var(--bg-dark); border-radius: 16px; border: 1px solid var(--border); }
    .stat-value { font-size: 1.75rem; font-weight: 700; margin: 8px 0; }
    .stat-label { font-size: 0.875rem; color: var(--text-secondary); }
    .chart-container { position: relative; height: 300px; margin-bottom: 24px; }
    .risk-score { text-align: center; padding: 24px; border-radius: 16px; font-size: 3rem; font-weight: 800; margin: 20px 0; }
    .risk-low { background: rgba(16, 185, 129, 0.2); color: var(--success); }
    .risk-medium { background: rgba(245, 158, 11, 0.2); color: var(--warning); }
    .risk-high { background: rgba(239, 68, 68, 0.2); color: var(--danger); }
  </style>
</head>
<body>
  <button class="theme-toggle" onclick="toggleTheme()">
    <span id="themeIcon">🌙</span>
    <span id="themeText">Dark Mode</span>
  </button>

  <div class="container">
    <a href="{{ url_for('index') }}" class="back-link">← Ana Sayfaya Dön</a>

    <div style="display:flex; justify-content:space-between; align-items:center; gap: 10px; margin-bottom: 20px;">
      <div>
        <h1 style="font-size: 2rem;">📊 Oturum Detayı</h1>
        <p style="color: var(--text-secondary);">{{ session_data.filename }}</p>
      </div>
      <div style="display:flex; gap:10px; flex-wrap:wrap;">
        {% if pdf_available %}
        <a href="{{ url_for('export_pdf', filename=session_data.filename) }}" style="padding:10px 18px; background:#ef4444; color:white; text-decoration:none; border-radius:10px; font-weight:700;">
          📄 PDF İndir
        </a>
        {% endif %}
        <a href="{{ url_for('compare_sessions') }}?session1={{ session_data.filename }}" style="padding:10px 18px; background:#3b82f6; color:white; text-decoration:none; border-radius:10px; font-weight:700;">
          📊 Karşılaştır
        </a>
      </div>
    </div>

    <div class="stats-grid">
      <div class="stat-item">
        <div class="stat-label">Süre</div>
        <div class="stat-value" style="color: var(--primary);">{{ stats.total_duration_seconds or 0 }}</div>
        <div class="stat-label">saniye</div>
      </div>
      <div class="stat-item">
        <div class="stat-label">Uykululuk</div>
        <div class="stat-value" style="color: var(--danger);">{{ "%.1f"|format(stats.drowsy_percentage or 0) }}%</div>
      </div>
      <div class="stat-item">
        <div class="stat-label">Dikkat Dağ.</div>
        <div class="stat-value" style="color: var(--warning);">{{ "%.1f"|format(stats.distracted_percentage or 0) }}%</div>
      </div>
      <div class="stat-item">
        <div class="stat-label">Göz Açıklığı</div>
        <div class="stat-value" style="color: var(--success);">{{ "%.2f"|format(stats.average_eye_openness or 0) }}</div>
      </div>
    </div>

    <div class="card">
      <h2 class="card-title">Risk Skoru</h2>
      <div class="risk-score {% if risk_score < 30 %}risk-low{% elif risk_score < 60 %}risk-medium{% else %}risk-high{% endif %}">
        {{ "%.0f"|format(risk_score) }}
      </div>
      <p style="text-align:center; color: var(--text-secondary);">
        {% if risk_score < 30 %} ✅ Düşük Risk - Güvenli Sürüş
        {% elif risk_score < 60 %} ⚠️ Orta Risk - Dikkatli Olun
        {% else %} 🔴 Yüksek Risk - Dikkat Gerekli
        {% endif %}
      </p>
    </div>

    <div class="card">
      <h2 class="card-title">Göz Açıklığı Trendi</h2>
      <div class="chart-container"><canvas id="eyeChart"></canvas></div>
    </div>

    <div class="card">
      <h2 class="card-title">Duygu Dağılımı</h2>
      <div class="chart-container"><canvas id="emotionChart"></canvas></div>
    </div>

    <div class="card">
      <h2 class="card-title">Uykululuk ve Dikkat Dağınıklığı Zaman Çizelgesi</h2>
      <div class="chart-container"><canvas id="timelineChart"></canvas></div>
    </div>

    <div class="card">
      <h2 class="card-title">Baş Pozisyonu (Head Yaw)</h2>
      <div class="chart-container"><canvas id="headYawChart"></canvas></div>
    </div>
  </div>

  <script>
    const records = {{ session_data.records | tojson }};
    const emotionDist = {{ stats.emotion_distribution | tojson }};

    new Chart(document.getElementById('eyeChart').getContext('2d'), {
      type: 'line',
      data: {
        labels: records.map((r, i) => i + 1),
        datasets: [{
          label: 'Göz Açıklığı',
          data: records.map(r => r.eye_openness || 0),
          borderColor: '#3b82f6',
          backgroundColor: 'rgba(59, 130, 246, 0.1)',
          tension: 0.4,
          fill: true
        }]
      },
      options: { responsive: true, maintainAspectRatio: false }
    });

    new Chart(document.getElementById('emotionChart').getContext('2d'), {
      type: 'doughnut',
      data: {
        labels: ['Normal', 'Yorgun'],
        datasets: [{
          data: [emotionDist.normal || 0, emotionDist.tired || 0],
          backgroundColor: ['#10b981', '#f59e0b']
        }]
      },
      options: { responsive: true, maintainAspectRatio: false }
    });

    new Chart(document.getElementById('timelineChart').getContext('2d'), {
      type: 'line',
      data: {
        labels: records.map((r, i) => i + 1),
        datasets: [{
          label: 'Uykululuk',
          data: records.map(r => r.drowsy ? 1 : 0),
          borderColor: '#ef4444',
          backgroundColor: 'rgba(239, 68, 68, 0.1)',
          tension: 0.4,
          fill: true
        }, {
          label: 'Dikkat Dağınıklığı',
          data: records.map(r => r.distracted ? 1 : 0),
          borderColor: '#f59e0b',
          backgroundColor: 'rgba(245, 158, 11, 0.1)',
          tension: 0.4,
          fill: true
        }]
      },
      options: { responsive: true, maintainAspectRatio: false }
    });

    new Chart(document.getElementById('headYawChart').getContext('2d'), {
      type: 'line',
      data: {
        labels: records.map((r, i) => i + 1),
        datasets: [{
          label: 'Baş Pozisyonu (Yaw)',
          data: records.map(r => r.head_yaw || 0),
          borderColor: '#8b5cf6',
          backgroundColor: 'rgba(139, 92, 246, 0.1)',
          tension: 0.4,
          fill: true
        }]
      },
      options: { responsive: true, maintainAspectRatio: false }
    });

    function toggleTheme() {
      const html = document.documentElement;
      const current = html.getAttribute('data-theme');
      const newTheme = current === 'light' ? 'dark' : 'light';
      html.setAttribute('data-theme', newTheme);
      localStorage.setItem('theme', newTheme);
      document.getElementById('themeIcon').textContent = newTheme === 'light' ? '☀️' : '🌙';
      document.getElementById('themeText').textContent = newTheme === 'light' ? 'Light Mode' : 'Dark Mode';
    }
    (function() {
      const saved = localStorage.getItem('theme') || 'dark';
      document.documentElement.setAttribute('data-theme', saved);
      document.getElementById('themeIcon').textContent = saved === 'light' ? '☀️' : '🌙';
      document.getElementById('themeText').textContent = saved === 'light' ? 'Light Mode' : 'Dark Mode';
    })();
  </script>
</body>
</html>
"""


COMPARE_TEMPLATE = """
<!doctype html>
<html lang="tr">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Oturum Karşılaştırma</title>
  <script src="https://cdn.jsdelivr.net/npm/chart.js@4.4.0/dist/chart.umd.min.js"></script>
  <style>
    * { margin:0; padding:0; box-sizing:border-box; }
    :root { --primary:#3b82f6; --success:#10b981; --danger:#ef4444; --warning:#f59e0b;
            --bg-dark:#0f172a; --bg-card:#1e293b; --text-primary:#f1f5f9; --text-secondary:#94a3b8; --border:#334155; }
    [data-theme="light"] { --bg-dark:#f8fafc; --bg-card:#ffffff; --text-primary:#0f172a; --text-secondary:#475569; --border:#e2e8f0; }
    body { font-family: Inter, system-ui, -apple-system, Segoe UI, Roboto, sans-serif; background: var(--bg-dark); color: var(--text-primary); padding: 20px; }
    .container { max-width: 1600px; margin: 0 auto; }
    .back-link { color: var(--primary); text-decoration: none; font-weight: 700; display:inline-block; margin-bottom: 10px; }
    .card { background: var(--bg-card); border-radius: 16px; padding: 24px; margin-bottom: 24px; border: 1px solid var(--border); }
    .card-title { font-size: 1.25rem; font-weight: 800; margin-bottom: 20px; }
    .select-form { display: grid; grid-template-columns: 1fr 1fr; gap: 20px; margin-bottom: 20px; }
    select { width: 100%; padding: 12px; border-radius: 10px; border: 2px solid var(--border); background: var(--bg-dark); color: var(--text-primary); }
    .comparison-grid { display:grid; grid-template-columns: 1fr 1fr; gap: 16px; }
    .stat-box { background: var(--bg-dark); padding: 16px; border-radius: 12px; border: 1px solid var(--border); text-align:center; }
    .stat-value { font-size: 1.5rem; font-weight: 900; margin: 8px 0; }
    .stat-label { font-size: .85rem; color: var(--text-secondary); }
    .chart-container { position: relative; height: 320px; }
    .better { color: var(--success); } .worse { color: var(--danger); } .equal { color: var(--text-secondary); }
  </style>
</head>
<body>
  <div class="container">
    <a href="{{ url_for('index') }}" class="back-link">← Ana Sayfaya Dön</a>
    <h1 style="font-size:2rem; margin: 10px 0 20px;">📊 Oturum Karşılaştırma</h1>

    <form method="get" action="{{ url_for('compare_sessions') }}" class="select-form">
      <div>
        <label style="display:block; margin-bottom:8px; color: var(--text-secondary); font-weight:700;">İlk Oturum</label>
        <select name="session1" required>
          <option value="">Seçiniz...</option>
          {% for s in sessions %}
            <option value="{{ s.filename }}" {% if selected_session1 == s.filename %}selected{% endif %}>{{ s.filename }}</option>
          {% endfor %}
        </select>
      </div>
      <div>
        <label style="display:block; margin-bottom:8px; color: var(--text-secondary); font-weight:700;">İkinci Oturum</label>
        <select name="session2" required>
          <option value="">Seçiniz...</option>
          {% for s in sessions %}
            <option value="{{ s.filename }}" {% if selected_session2 == s.filename %}selected{% endif %}>{{ s.filename }}</option>
          {% endfor %}
        </select>
      </div>
      <div style="grid-column:1/-1;">
        <button type="submit" style="padding: 12px 24px; background: var(--primary); color:white; border:none; border-radius: 10px; font-weight: 900; cursor:pointer;">
          Karşılaştır
        </button>
      </div>
    </form>

    {% if session1_data and session2_data %}
    <div class="card">
      <h2 class="card-title">Genel Karşılaştırma</h2>
      <div class="comparison-grid">
        <div class="stat-box">
          <div class="stat-label">Süre (sn)</div>
          <div class="stat-value" style="color: var(--primary);">{{ stats1.total_duration_seconds or 0 }}</div>
        </div>
        <div class="stat-box">
          <div class="stat-label">Süre (sn)</div>
          <div class="stat-value" style="color: var(--primary);">{{ stats2.total_duration_seconds or 0 }}</div>
        </div>

        <div class="stat-box">
          <div class="stat-label">Uykululuk</div>
          <div class="stat-value {% if (stats1.drowsy_percentage or 0) < (stats2.drowsy_percentage or 0) %}better{% elif (stats1.drowsy_percentage or 0) > (stats2.drowsy_percentage or 0) %}worse{% else %}equal{% endif %}">
            {{ "%.1f"|format(stats1.drowsy_percentage or 0) }}%
          </div>
        </div>
        <div class="stat-box">
          <div class="stat-label">Uykululuk</div>
          <div class="stat-value {% if (stats2.drowsy_percentage or 0) < (stats1.drowsy_percentage or 0) %}better{% elif (stats2.drowsy_percentage or 0) > (stats1.drowsy_percentage or 0) %}worse{% else %}equal{% endif %}">
            {{ "%.1f"|format(stats2.drowsy_percentage or 0) }}%
          </div>
        </div>

        <div class="stat-box">
          <div class="stat-label">Dikkat Dağ.</div>
          <div class="stat-value {% if (stats1.distracted_percentage or 0) < (stats2.distracted_percentage or 0) %}better{% elif (stats1.distracted_percentage or 0) > (stats2.distracted_percentage or 0) %}worse{% else %}equal{% endif %}">
            {{ "%.1f"|format(stats1.distracted_percentage or 0) }}%
          </div>
        </div>
        <div class="stat-box">
          <div class="stat-label">Dikkat Dağ.</div>
          <div class="stat-value {% if (stats2.distracted_percentage or 0) < (stats1.distracted_percentage or 0) %}better{% elif (stats2.distracted_percentage or 0) > (stats1.distracted_percentage or 0) %}worse{% else %}equal{% endif %}">
            {{ "%.1f"|format(stats2.distracted_percentage or 0) }}%
          </div>
        </div>

        <div class="stat-box">
          <div class="stat-label">Risk Skoru</div>
          <div class="stat-value {% if risk1 < risk2 %}better{% elif risk1 > risk2 %}worse{% else %}equal{% endif %}">{{ "%.0f"|format(risk1) }}</div>
        </div>
        <div class="stat-box">
          <div class="stat-label">Risk Skoru</div>
          <div class="stat-value {% if risk2 < risk1 %}better{% elif risk2 > risk1 %}worse{% else %}equal{% endif %}">{{ "%.0f"|format(risk2) }}</div>
        </div>
      </div>
    </div>

    <div class="card">
      <h2 class="card-title">Metrik Karşılaştırması</h2>
      <div class="chart-container"><canvas id="comparisonChart"></canvas></div>
    </div>

    <div class="card">
      <h2 class="card-title">Duygu Dağılımı (Normal/Yorgun)</h2>
      <div class="chart-container"><canvas id="emotionChart"></canvas></div>
    </div>

    <script>
      const compCtx = document.getElementById('comparisonChart').getContext('2d');
      new Chart(compCtx, {
        type: 'bar',
        data: {
          labels: ['Uykululuk %', 'Dikkat Dağ. %', 'Risk Skoru', 'Göz Açıklığı x100'],
          datasets: [{
            label: '{{ selected_session1 }}',
            data: [{{ stats1.drowsy_percentage or 0 }}, {{ stats1.distracted_percentage or 0 }}, {{ risk1 }}, {{ (stats1.average_eye_openness or 0) * 100 }}],
            backgroundColor: 'rgba(59,130,246,0.55)',
            borderColor: '#3b82f6',
            borderWidth: 2
          },{
            label: '{{ selected_session2 }}',
            data: [{{ stats2.drowsy_percentage or 0 }}, {{ stats2.distracted_percentage or 0 }}, {{ risk2 }}, {{ (stats2.average_eye_openness or 0) * 100 }}],
            backgroundColor: 'rgba(16,185,129,0.55)',
            borderColor: '#10b981',
            borderWidth: 2
          }]
        },
        options: { responsive:true, maintainAspectRatio:false }
      });

      const emoCtx = document.getElementById('emotionChart').getContext('2d');
      const emo1 = {{ stats1.emotion_distribution | tojson }};
      const emo2 = {{ stats2.emotion_distribution | tojson }};
      new Chart(emoCtx, {
        type: 'bar',
        data: {
          labels: ['Normal', 'Yorgun'],
          datasets: [{
            label: '{{ selected_session1 }}',
            data: [emo1.normal || 0, emo1.tired || 0],
            backgroundColor: 'rgba(59,130,246,0.55)',
            borderColor: '#3b82f6',
            borderWidth: 2
          },{
            label: '{{ selected_session2 }}',
            data: [emo2.normal || 0, emo2.tired || 0],
            backgroundColor: 'rgba(16,185,129,0.55)',
            borderColor: '#10b981',
            borderWidth: 2
          }]
        },
        options: { responsive:true, maintainAspectRatio:false }
      });
    </script>
    {% endif %}
  </div>
</body>
</html>
"""



In [14]:
@app.route("/", methods=["GET"])
def index():
    ingest_root_sessions_to_folder()
    
    # HTML oluştur
    html = render_template_string(
        PAGE_TEMPLATE,
        profile=load_profile_info(),
        sessions=list_sessions(),
        dashboard_stats=calculate_dashboard_stats(),
        contacts=load_emergency_contacts(),
        contact_error=None,
        contact_success=None,
        start_error=None,
        start_success=None,
    )
    
    # Chat widget'ı ekle
    html = inject_chat_widget_to_html(html)
    
    return html


@app.route("/save_profile", methods=["POST"])
def save_profile():
    save_profile_info(
        request.form.get("name", ""),
        request.form.get("surname", ""),
        request.form.get("plate", ""),
    )
    return redirect(url_for("index"))


@app.route("/reset_profile", methods=["POST"])
def reset_profile_route():
    reset_profile()
    return redirect(url_for("index"))


@app.route("/delete_sessions", methods=["POST"])
def delete_sessions_route():
    delete_all_sessions()
    return redirect(url_for("index"))


@app.route("/start_drive", methods=["POST"])
def start_drive():
    profile = load_profile_info()

    if not profile.get("name") or not profile.get("surname") or not profile.get("plate"):
        html = render_template_string(
            PAGE_TEMPLATE,
            profile=profile,
            sessions=list_sessions(),
            dashboard_stats=calculate_dashboard_stats(),
            contacts=load_emergency_contacts(),
            contact_error=None,
            contact_success=None,
            start_error="❌ Lütfen önce ad, soyad ve plaka bilgisini doldurun.",
            start_success=None,
        )
        return inject_chat_widget_to_html(html)

    def run_system():
        try:
            import time
            time.sleep(0.8)
            try:
                system = FullDriverMonitoringSystem(
                    model_path="emotion_model.pkl",
                    norm_path="normalization_params.npz",
                )
            except TypeError:
                system = FullDriverMonitoringSystem()

            system.run(camera_id=0)
            ingest_root_sessions_to_folder()
        except Exception as e:
            print("[HATA] Sistem çalıştırılamadı:", e)
            import traceback
            traceback.print_exc()

    import threading
    t = threading.Thread(target=run_system, daemon=True)
    t.start()

    html = render_template_string(
        PAGE_TEMPLATE,
        profile=profile,
        sessions=list_sessions(),
        dashboard_stats=calculate_dashboard_stats(),
        contacts=load_emergency_contacts(),
        contact_error=None,
        contact_success=None,
        start_error=None,
        start_success="✅ Sürüş simülasyonu başlatıldı!",
    )
    return inject_chat_widget_to_html(html)


@app.route("/sessions/<path:filename>")
def download_session(filename):
    return send_from_directory(SESSIONS_DIR, filename, as_attachment=True)


@app.route("/session/<path:filename>")
def session_detail(filename):
    session_data = get_session_detail(filename)
    if not session_data:
        return "Oturum bulunamadı", 404

    stats = session_data.get("statistics", {}) or {}
    risk_score = calculate_risk_score(stats)

    return render_template_string(
        SESSION_DETAIL_TEMPLATE,
        session_data={"filename": filename, "records": session_data.get("records", [])},
        stats=stats,
        risk_score=risk_score,
        pdf_available=PDF_AVAILABLE,
    )


@app.route("/export_pdf/<path:filename>")
def export_pdf(filename):
    session_data = get_session_detail(filename)
    if not session_data:
        return "Oturum bulunamadı", 404

    pdf_buffer = generate_pdf_report(session_data, filename)
    if not pdf_buffer:
        return "PDF oluşturulamadı. reportlab yüklü olmalıdır.", 500

    pdf_filename = filename.replace(".json", ".pdf")
    return send_file(
        pdf_buffer,
        mimetype="application/pdf",
        as_attachment=True,
        download_name=pdf_filename,
    )


@app.route("/compare")
def compare_sessions():
    sessions = list_sessions()
    selected_session1 = request.args.get("session1", "")
    selected_session2 = request.args.get("session2", "")

    session1_data = None
    session2_data = None
    stats1 = {}
    stats2 = {}
    risk1 = 0
    risk2 = 0

    if selected_session1:
        session1_data = get_session_detail(selected_session1)
        if session1_data:
            stats1 = session1_data.get("statistics", {}) or {}
            risk1 = calculate_risk_score(stats1)

    if selected_session2:
        session2_data = get_session_detail(selected_session2)
        if session2_data:
            stats2 = session2_data.get("statistics", {}) or {}
            risk2 = calculate_risk_score(stats2)

    return render_template_string(
        COMPARE_TEMPLATE,
        sessions=sessions,
        selected_session1=selected_session1,
        selected_session2=selected_session2,
        session1_data=session1_data,
        session2_data=session2_data,
        stats1=stats1,
        stats2=stats2,
        risk1=risk1,
        risk2=risk2,
    )


@app.route("/contacts/add", methods=["POST"])
def add_contact():
    new_item, err = add_emergency_contact(
        request.form.get("contact_name", ""),
        request.form.get("contact_chat_id", ""),
    )
    html = render_template_string(
        PAGE_TEMPLATE,
        profile=load_profile_info(),
        sessions=list_sessions(),
        dashboard_stats=calculate_dashboard_stats(),
        contacts=load_emergency_contacts(),
        contact_error=err,
        contact_success=("✅ Kişi eklendi" if new_item else None),
        start_error=None,
        start_success=None,
    )
    return inject_chat_widget_to_html(html)


@app.route("/contacts/delete/<path:contact_id>", methods=["POST"])
def delete_contact_route(contact_id):
    removed = delete_emergency_contact(contact_id)
    return jsonify({"ok": True, "removed": removed})


@app.route("/api/telegram/test", methods=["GET"])
def telegram_test():
    return jsonify(test_telegram_connection())


@app.route("/api/telegram/get_chat_id", methods=["GET"])
def get_telegram_chat_id_help():
    token = os.getenv("TELEGRAM_BOT_TOKEN", "").strip()
    if not token:
        return jsonify({"ok": False, "error": "Bot token tanımlı değil"})
    return jsonify({"ok": True, "getUpdates_url": f"https://api.telegram.org/bot{token}/getUpdates"})


@app.route("/api/emergency/trigger", methods=["POST"])
def emergency_trigger():
    payload = request.get_json(silent=True) or {}
    reason = (payload.get("reason") or "Acil durum").strip()
    seconds = payload.get("seconds", None)

    contacts = load_emergency_contacts()
    if not contacts:
        return jsonify({"ok": False, "error": "Acil kişi yok", "sent": 0}), 400

    msg = build_emergency_message(reason=reason, seconds=seconds, extra=payload)

    results = []
    sent = 0
    for c in contacts:
        chat_id = c.get("chat_id")
        ok, detail = send_telegram_message(chat_id, msg)
        results.append({"contact": c.get("name"), "chat_id": chat_id, "ok": ok, "detail": detail})
        if ok:
            sent += 1

    return jsonify({"ok": True, "sent": sent, "total": len(contacts), "results": results})

In [15]:

# AI CHATBOT Fonksiyonları

import os
from datetime import datetime

# Anthropic API kurulumu
try:
    from anthropic import Anthropic
    ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY", "").strip()
    if ANTHROPIC_API_KEY:
        anthropic_client = Anthropic(api_key=ANTHROPIC_API_KEY)
        AI_AVAILABLE = True
    else:
        AI_AVAILABLE = False
        print("[UYARI] ANTHROPIC_API_KEY bulunamadı. AI Chat devre dışı.")
except ImportError:
    AI_AVAILABLE = False
    print("[UYARI] anthropic kütüphanesi yüklü değil. pip install anthropic")

# Global değişkenler
chatbot_sessions_dir = None
chatbot_conversation_history = []

def init_chatbot(sessions_dir):
    """AI Chatbot'u başlat"""
    global chatbot_sessions_dir
    chatbot_sessions_dir = sessions_dir
    print("[OK] AI Chatbot başlatıldı ✓")

def get_all_sessions_summary():
    """Tüm oturumların özetini al"""
    if not chatbot_sessions_dir:
        return "Oturum dizini ayarlanmamış."
    
    sessions = list_sessions()
    if not sessions:
        return "Henüz kayıtlı oturum yok."
    
    summary = f"Toplam {len(sessions)} oturum var:\n\n"
    for i, session in enumerate(sessions[:10], 1):
        stats = session.get('stats', {})
        summary += f"{i}. {session.get('filename', 'N/A')}\n"
        summary += f"   - Başlangıç: {session.get('start_time', 'N/A')[:19]}\n"
        summary += f"   - Uykululuk: {stats.get('drowsy_percentage', 0):.1f}%\n"
        summary += f"   - Dikkat Dağınıklığı: {stats.get('distracted_percentage', 0):.1f}%\n\n"
    
    if len(sessions) > 10:
        summary += f"... ve {len(sessions) - 10} oturum daha."
    
    return summary
# AI Chatbot hücresinde chat_with_ai fonksiyonunu bulun
# ve model adını güncelleyin:

def chat_with_ai(user_query, query_type="general", sessions=None):
    """AI ile sohbet et"""
    global chatbot_conversation_history
    
    if not AI_AVAILABLE:
        return {
            "ok": False,
            "error": "AI hizmeti kullanılamıyor. ANTHROPIC_API_KEY ayarlanmamış veya kütüphane yüklü değil."
        }
    
    try:
        # Sistem mesajı
        system_message = """Sen bir sürücü güvenlik asistanısın. 
Kullanıcının sürüş verilerini analiz edip öneriler sunuyorsun.
Kısa ve net cevaplar ver."""
        
        # Bağlam ekle
        context = ""
        if query_type == "general":
            context = get_all_sessions_summary()
        elif query_type == "compare" and sessions:
            context = f"Karşılaştırma yapılacak oturumlar: {', '.join(sessions)}"
        
        # Mesaj oluştur
        user_message = f"{context}\n\nKullanıcı sorusu: {user_query}"
        
        # Sohbet geçmişine ekle
        chatbot_conversation_history.append({
            "role": "user",
            "content": user_message
        })
        
        # API çağrısı - DOĞRU MODEL ADI
        response = anthropic_client.messages.create(
            model="claude-sonnet-4-20250514",  # ← BU SATIRI DEĞİŞTİRİN!
            max_tokens=1000,
            system=system_message,
            messages=chatbot_conversation_history[-10:]  # Son 10 mesaj
        )
        
        assistant_reply = response.content[0].text
        
        # Sohbet geçmişine ekle
        chatbot_conversation_history.append({
            "role": "assistant",
            "content": assistant_reply
        })
        
        return {
            "ok": True,
            "response": assistant_reply
        }
        
    except Exception as e:
        return {
            "ok": False,
            "error": f"AI hatası: {str(e)}"
        }

def clear_chat_history():
    """Sohbet geçmişini temizle"""
    global chatbot_conversation_history
    chatbot_conversation_history = []
    return {"ok": True}

def setup_ai_routes(app, list_sessions_func):
    """AI Chat route'larını ekle"""
    
    @app.route("/api/ai_chat", methods=["POST"])
    def ai_chat_endpoint():
        data = request.get_json() or {}
        query = data.get("query", "").strip()
        query_type = data.get("type", "general")
        sessions = data.get("sessions", [])
        
        if not query:
            return jsonify({"ok": False, "error": "Soru boş olamaz"})
        
        result = chat_with_ai(query, query_type, sessions)
        return jsonify(result)
    
    @app.route("/api/ai_chat/clear", methods=["POST"])
    def ai_chat_clear():
        result = clear_chat_history()
        return jsonify(result)

In [16]:
# Hem Proje Rehberi Hem Veri Analizi
AI_CHAT_WIDGET_HTML = """
<!-- AI Chat Widget CSS ve HTML -->
<style>
    /* Chat Widget Container */
    .chat-widget-container {
        position: fixed;
        bottom: 20px;
        right: 20px;
        z-index: 9999;
        font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
    }
    
    /* Chat Button */
    .chat-widget-button {
        width: 60px;
        height: 60px;
        border-radius: 50%;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        border: none;
        cursor: pointer;
        box-shadow: 0 4px 12px rgba(0,0,0,0.15);
        display: flex;
        align-items: center;
        justify-content: center;
        transition: all 0.3s ease;
        position: relative;
    }
    
    .chat-widget-button:hover {
        transform: scale(1.1);
        box-shadow: 0 6px 20px rgba(0,0,0,0.2);
    }
    
    .chat-widget-button.active {
        background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
    }
    
    .chat-widget-button svg {
        width: 28px;
        height: 28px;
        fill: white;
    }
    
    /* Notification Badge */
    .chat-widget-badge {
        position: absolute;
        top: -5px;
        right: -5px;
        background: #ff4444;
        color: white;
        border-radius: 50%;
        width: 20px;
        height: 20px;
        display: flex;
        align-items: center;
        justify-content: center;
        font-size: 11px;
        font-weight: bold;
        display: none;
    }
    
    /* Chat Window */
    .chat-widget-window {
        position: absolute;
        bottom: 80px;
        right: 0;
        width: 400px;
        height: 600px;
        background: #1a1a2e;
        border-radius: 16px;
        box-shadow: 0 8px 32px rgba(0,0,0,0.3);
        display: none;
        flex-direction: column;
        overflow: hidden;
        animation: slideUp 0.3s ease;
    }
    
    .chat-widget-window.active {
        display: flex;
    }
    
    @keyframes slideUp {
        from {
            opacity: 0;
            transform: translateY(20px);
        }
        to {
            opacity: 1;
            transform: translateY(0);
        }
    }
    
    /* Chat Header */
    .chat-widget-header {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        padding: 16px 20px;
        display: flex;
        justify-content: space-between;
        align-items: center;
    }
    
    .chat-widget-header h3 {
        margin: 0;
        font-size: 16px;
        font-weight: 600;
    }
    
    .chat-widget-header small {
        display: block;
        font-size: 11px;
        opacity: 0.9;
        margin-top: 2px;
    }
    
    .chat-widget-close {
        background: none;
        border: none;
        color: white;
        cursor: pointer;
        font-size: 24px;
        padding: 0;
        width: 28px;
        height: 28px;
        display: flex;
        align-items: center;
        justify-content: center;
        border-radius: 4px;
        transition: background 0.2s;
    }
    
    .chat-widget-close:hover {
        background: rgba(255,255,255,0.1);
    }
    
    /* Chat Messages */
    .chat-widget-messages {
        flex: 1;
        overflow-y: auto;
        padding: 20px;
        background: #16213e;
    }
    
    .chat-widget-message {
        margin-bottom: 16px;
        animation: fadeIn 0.3s ease;
    }
    
    @keyframes fadeIn {
        from { opacity: 0; transform: translateY(10px); }
        to { opacity: 1; transform: translateY(0); }
    }
    
    .chat-widget-message-bubble {
        display: inline-block;
        max-width: 85%;
        padding: 12px 16px;
        border-radius: 12px;
        font-size: 14px;
        line-height: 1.5;
        word-wrap: break-word;
        white-space: pre-wrap;
    }
    
    .chat-widget-message.user {
        text-align: right;
    }
    
    .chat-widget-message.user .chat-widget-message-bubble {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        border-bottom-right-radius: 4px;
    }
    
    .chat-widget-message.assistant .chat-widget-message-bubble {
        background: #0f3460;
        color: #e0e0e0;
        border-bottom-left-radius: 4px;
    }
    
    .chat-widget-message.system .chat-widget-message-bubble {
        background: rgba(255,193,7,0.2);
        color: #ffc107;
        border-left: 3px solid #ffc107;
    }
    
    /* Loading */
    .chat-widget-loading {
        display: none;
        text-align: center;
        padding: 12px;
    }
    
    .chat-widget-loading.active {
        display: block;
    }
    
    .chat-widget-loading-dots {
        display: inline-block;
    }
    
    .chat-widget-loading-dots span {
        display: inline-block;
        width: 8px;
        height: 8px;
        border-radius: 50%;
        background: #667eea;
        margin: 0 3px;
        animation: bounce 1.4s infinite ease-in-out both;
    }
    
    .chat-widget-loading-dots span:nth-child(1) { animation-delay: -0.32s; }
    .chat-widget-loading-dots span:nth-child(2) { animation-delay: -0.16s; }
    
    @keyframes bounce {
        0%, 80%, 100% { transform: scale(0); }
        40% { transform: scale(1); }
    }
    
    /* Chat Input */
    .chat-widget-input-area {
        padding: 16px;
        background: #1a1a2e;
        border-top: 1px solid rgba(255,255,255,0.1);
    }
    
    .chat-widget-input-wrapper {
        display: flex;
        gap: 8px;
    }
    
    .chat-widget-input {
        flex: 1;
        background: #0f3460;
        border: 1px solid rgba(255,255,255,0.1);
        border-radius: 24px;
        padding: 12px 16px;
        color: white;
        font-size: 14px;
        outline: none;
        transition: border-color 0.2s;
    }
    
    .chat-widget-input:focus {
        border-color: #667eea;
    }
    
    .chat-widget-input::placeholder {
        color: rgba(255,255,255,0.5);
    }
    
    .chat-widget-send {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        border: none;
        border-radius: 50%;
        width: 44px;
        height: 44px;
        cursor: pointer;
        display: flex;
        align-items: center;
        justify-content: center;
        transition: transform 0.2s;
        flex-shrink: 0;
    }
    
    .chat-widget-send:hover:not(:disabled) {
        transform: scale(1.1);
    }
    
    .chat-widget-send:disabled {
        opacity: 0.5;
        cursor: not-allowed;
    }
    
    .chat-widget-send svg {
        width: 20px;
        height: 20px;
        fill: white;
    }
    
    /* Quick Questions */
    .chat-widget-quick-questions {
        padding: 0 20px 12px;
        display: flex;
        flex-wrap: wrap;
        gap: 6px;
        max-height: 120px;
        overflow-y: auto;
    }
    
    .chat-widget-quick-btn {
        background: rgba(102, 126, 234, 0.2);
        border: 1px solid rgba(102, 126, 234, 0.4);
        color: #667eea;
        padding: 6px 10px;
        border-radius: 14px;
        font-size: 11px;
        cursor: pointer;
        transition: all 0.2s;
        white-space: nowrap;
    }
    
    .chat-widget-quick-btn:hover {
        background: rgba(102, 126, 234, 0.3);
        border-color: #667eea;
    }
    
    /* Scrollbar */
    .chat-widget-messages::-webkit-scrollbar,
    .chat-widget-quick-questions::-webkit-scrollbar {
        width: 6px;
    }
    
    .chat-widget-messages::-webkit-scrollbar-track,
    .chat-widget-quick-questions::-webkit-scrollbar-track {
        background: transparent;
    }
    
    .chat-widget-messages::-webkit-scrollbar-thumb,
    .chat-widget-quick-questions::-webkit-scrollbar-thumb {
        background: rgba(102, 126, 234, 0.5);
        border-radius: 3px;
    }
    
    .chat-widget-messages::-webkit-scrollbar-thumb:hover,
    .chat-widget-quick-questions::-webkit-scrollbar-thumb:hover {
        background: rgba(102, 126, 234, 0.7);
    }
    
    /* Responsive */
    @media (max-width: 480px) {
        .chat-widget-window {
            width: calc(100vw - 40px);
            height: calc(100vh - 120px);
            bottom: 80px;
            right: 20px;
            left: 20px;
        }
    }
</style>

<!-- Chat Widget HTML -->
<div class="chat-widget-container">
    <button class="chat-widget-button" id="chatWidgetBtn" onclick="toggleChatWidget()">
        <svg viewBox="0 0 24 24">
            <path d="M12 2C6.48 2 2 6.48 2 12c0 1.54.36 3 .97 4.29L2 22l5.71-.97C9 21.64 10.46 22 12 22c5.52 0 10-4.48 10-10S17.52 2 12 2zm0 18c-1.38 0-2.68-.29-3.88-.8l-.28-.12-2.9.49.49-2.9-.12-.28C4.79 14.68 4.5 13.38 4.5 12c0-4.14 3.36-7.5 7.5-7.5s7.5 3.36 7.5 7.5-3.36 7.5-7.5 7.5z"/>
            <circle cx="8.5" cy="12" r="1"/>
            <circle cx="12" cy="12" r="1"/>
            <circle cx="15.5" cy="12" r="1"/>
        </svg>
        <span class="chat-widget-badge" id="chatWidgetBadge">1</span>
    </button>
    
    <div class="chat-widget-window" id="chatWidgetWindow">
        <div class="chat-widget-header">
            <div>
                <h3>🤖 AI Asistan</h3>
                <small>Proje rehberi & Veri analizi</small>
            </div>
            <button class="chat-widget-close" onclick="toggleChatWidget()">×</button>
        </div>
        
        <div class="chat-widget-messages" id="chatWidgetMessages">
            <div class="chat-widget-message system">
                <div class="chat-widget-message-bubble">
                    👋 Merhaba! Hem proje hakkında hem verileriniz hakkında soru sorabilirsiniz.
                </div>
            </div>
        </div>
        
        <div class="chat-widget-quick-questions">
            <button class="chat-widget-quick-btn" onclick="quickAskWidget('Son 5 oturumda dikkat dağınıklığı ortalaması?')">
                📊 Son 5 Oturum
            </button>
            <button class="chat-widget-quick-btn" onclick="quickAskWidget('En iyi ve en kötü oturumum?')">
                🏆 En İyi/Kötü
            </button>
            <button class="chat-widget-quick-btn" onclick="quickAskWidget('Performansım nasıl?')">
                📈 Performans
            </button>
            <button class="chat-widget-quick-btn" onclick="quickAskWidget('Oturum karşılaştırma nasıl yapılır?')">
                ⚖️ Karşılaştırma
            </button>
            <button class="chat-widget-quick-btn" onclick="quickAskWidget('PDF nasıl oluşturulur?')">
                📄 PDF
            </button>
            <button class="chat-widget-quick-btn" onclick="quickAskWidget('Sürüşü nasıl başlatırım?')">
                🚗 Başlat
            </button>
            <button class="chat-widget-quick-btn" onclick="quickAskWidget('Telegram nasıl kurulur?')">
                📱 Telegram
            </button>
            <button class="chat-widget-quick-btn" onclick="quickAskWidget('Tüm özellikleri göster')">
                ✨ Özellikler
            </button>
        </div>
        
        <div class="chat-widget-loading" id="chatWidgetLoading">
            <div class="chat-widget-loading-dots">
                <span></span>
                <span></span>
                <span></span>
            </div>
        </div>
        
        <div class="chat-widget-input-area">
            <div class="chat-widget-input-wrapper">
                <input 
                    type="text" 
                    class="chat-widget-input" 
                    id="chatWidgetInput" 
                    placeholder="Bir soru sorun..."
                    onkeypress="if(event.key==='Enter') sendWidgetMessage()"
                >
                <button class="chat-widget-send" id="chatWidgetSend" onclick="sendWidgetMessage()">
                    <svg viewBox="0 0 24 24">
                        <path d="M2.01 21L23 12 2.01 3 2 10l15 2-15 2z"/>
                    </svg>
                </button>
            </div>
        </div>
    </div>
</div>

<script>
let chatWidgetOpen = false;

function toggleChatWidget() {
    const window = document.getElementById('chatWidgetWindow');
    const button = document.getElementById('chatWidgetBtn');
    const badge = document.getElementById('chatWidgetBadge');
    
    chatWidgetOpen = !chatWidgetOpen;
    
    if (chatWidgetOpen) {
        window.classList.add('active');
        button.classList.add('active');
        badge.style.display = 'none';
        document.getElementById('chatWidgetInput').focus();
        scrollWidgetToBottom();
    } else {
        window.classList.remove('active');
        button.classList.remove('active');
    }
}

function addWidgetMessage(text, type) {
    const messagesDiv = document.getElementById('chatWidgetMessages');
    const messageDiv = document.createElement('div');
    messageDiv.className = `chat-widget-message ${type}`;
    
    const bubble = document.createElement('div');
    bubble.className = 'chat-widget-message-bubble';
    bubble.textContent = text;
    
    messageDiv.appendChild(bubble);
    messagesDiv.appendChild(messageDiv);
    
    scrollWidgetToBottom();
}

function scrollWidgetToBottom() {
    const messagesDiv = document.getElementById('chatWidgetMessages');
    setTimeout(() => {
        messagesDiv.scrollTop = messagesDiv.scrollHeight;
    }, 100);
}

function setWidgetLoading(loading) {
    const loadingDiv = document.getElementById('chatWidgetLoading');
    const sendBtn = document.getElementById('chatWidgetSend');
    const input = document.getElementById('chatWidgetInput');
    
    if (loading) {
        loadingDiv.classList.add('active');
        sendBtn.disabled = true;
        input.disabled = true;
    } else {
        loadingDiv.classList.remove('active');
        sendBtn.disabled = false;
        input.disabled = false;
    }
}

async function sendWidgetMessage() {
    const input = document.getElementById('chatWidgetInput');
    const message = input.value.trim();
    
    if (!message) return;
    
    addWidgetMessage(message, 'user');
    input.value = '';
    setWidgetLoading(true);
    
    try {
        const response = await fetch('/api/ai_chat', {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({ 
                query: message, 
                type: 'general' 
            })
        });
        
        const data = await response.json();
        
        if (data.ok) {
            addWidgetMessage(data.response, 'assistant');
        } else {
            addWidgetMessage('❌ Hata: ' + data.error, 'system');
        }
    } catch (error) {
        addWidgetMessage('❌ Bağlantı hatası: ' + error.message, 'system');
    } finally {
        setWidgetLoading(false);
        input.focus();
    }
}

function quickAskWidget(question) {
    document.getElementById('chatWidgetInput').value = question;
    sendWidgetMessage();
}
</script>
"""

# inject_chat_widget_to_html fonksiyonu
def inject_chat_widget_to_html(html_content):
    """HTML'e chat widget ekle"""
    if '</body>' in html_content:
        return html_content.replace('</body>', AI_CHAT_WIDGET_HTML + '\n</body>')
    else:
        return html_content + AI_CHAT_WIDGET_HTML

# AI CHATBOT BACKEND - TAM ÖZELLİKLİ 

import os
from datetime import datetime

# Anthropic API
try:
    from anthropic import Anthropic
    ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY", "").strip()
    if ANTHROPIC_API_KEY:
        anthropic_client = Anthropic(api_key=ANTHROPIC_API_KEY)
        AI_AVAILABLE = True
    else:
        AI_AVAILABLE = False
        print("[UYARI] ANTHROPIC_API_KEY bulunamadı. AI Chat devre dışı.")
except ImportError:
    AI_AVAILABLE = False
    print("[UYARI] anthropic kütüphanesi yüklü değil.")

# Global değişkenler
chatbot_sessions_dir = None
chatbot_conversation_history = []
_list_sessions_func = None  # ← DÜZELTİLDİ: Fonksiyonu saklayacak değişken

# Proje rehberi bilgileri
PROJECT_GUIDE = """
## PROJE ÖZELLİKLERİ:

1. **Gerçek Zamanlı Sürücü İzleme**: Webcam, göz takibi, baş pozisyonu analizi
2. **AI Duygu Analizi**: Yorgunluk, uykululuk tespiti, risk skoru
3. **Telegram Bildirimleri**: Acil durum bildirimleri, bot entegrasyonu
4. **Oturum Karşılaştırma**: İki oturumu yan yana karşılaştırma
5. **PDF Rapor**: Detaylı sürüş raporu oluşturma
6. **Modern Arayüz**: Dark/Light tema, responsive tasarım

## KULLANIM:

**Sürüş Başlatma:**
1. Ad, soyad, plaka girin
2. "Sürüşü Başlat" butonu
3. ESC ile durdurun

**Oturum Karşılaştırma:**
1. "Karşılaştır" butonuna tıklayın veya /compare
2. 2 oturum seçin
3. "Karşılaştır" butonu

**PDF İndirme:**
- Oturum listesinde "PDF" butonu
- Otomatik indirilir

**Telegram Kurulum:**
1. @akilli_surucu_bot'u bulun
2. /start yazın
3. "Chat ID Al" butonu
4. ID'yi kopyalayın
5. Acil kişiler bölümüne ekleyin

**Risk Skoru:**
Risk = (Uykululuk × 0.45) + (Dikkat × 0.45) + (Yorgunluk × 0.10)
"""

def get_sessions_data():
    """Oturum verilerini al - DÜZELTİLDİ"""
    global _list_sessions_func
    try:
        if _list_sessions_func and callable(_list_sessions_func):  # ← Kontrol eklendi
            return _list_sessions_func()
        else:
            print("[UYARI] list_sessions fonksiyonu henüz ayarlanmadı")
            return []
    except Exception as e:
        print(f"[HATA] Oturum verileri alınamadı: {e}")
        return []

def analyze_session_data(sessions_list, query):
    """Oturum verilerini analiz et"""
    if not sessions_list or len(sessions_list) == 0:
        return "❌ Henüz hiç oturum kaydı yok. 'Sürüşü Başlat' butonu ile ilk oturumunuzu oluşturun!"
    
    # Genel özet
    summary = f"📊 TOPLAM {len(sessions_list)} OTURUM\n\n"
    
    # Son 5 oturumu göster
    display_count = min(5, len(sessions_list))
    summary += f"SON {display_count} OTURUM:\n\n"
    
    for i, s in enumerate(sessions_list[:display_count], 1):
        stats = s.get('stats', {})
        summary += f"[{i}] {s.get('filename', 'N/A')}\n"
        summary += f"    ⏱ Süre: {int(stats.get('total_duration_seconds', 0)/60)} dk\n"
        summary += f"    😴 Uyku: {stats.get('drowsy_percentage', 0):.1f}%\n"
        summary += f"    👀 Dikkat: {stats.get('distracted_percentage', 0):.1f}%\n"
        summary += f"    👁 Kırpma: {stats.get('total_blinks', 0)}\n\n"
    
    # Ortalamalar
    avg_drowsy = sum(s.get('stats', {}).get('drowsy_percentage', 0) for s in sessions_list) / len(sessions_list)
    avg_dist = sum(s.get('stats', {}).get('distracted_percentage', 0) for s in sessions_list) / len(sessions_list)
    total_min = int(sum(s.get('stats', {}).get('total_duration_seconds', 0) for s in sessions_list) / 60)
    
    summary += f"📈 ORTALAMALAR:\n"
    summary += f"- Toplam Süre: {total_min} dakika\n"
    summary += f"- Ort. Uyku: {avg_drowsy:.1f}%\n"
    summary += f"- Ort. Dikkat: {avg_dist:.1f}%\n\n"
    
    # En iyi/kötü
    if len(sessions_list) > 1:
        best = min(sessions_list, key=lambda x: x.get('stats', {}).get('drowsy_percentage', 100) + x.get('stats', {}).get('distracted_percentage', 100))
        worst = max(sessions_list, key=lambda x: x.get('stats', {}).get('drowsy_percentage', 0) + x.get('stats', {}).get('distracted_percentage', 0))
        
        summary += f"🏆 EN İYİ: {best.get('filename', 'N/A')}\n"
        summary += f"⚠️ EN RİSKLİ: {worst.get('filename', 'N/A')}\n"
    
    return summary

def init_chatbot(sessions_dir):
    """AI Chatbot'u başlat"""
    global chatbot_sessions_dir
    chatbot_sessions_dir = sessions_dir
    print("[OK] AI Chatbot (Proje Rehberi + Veri Analizi) başlatıldı ✓")

def chat_with_ai(user_query, query_type="general", sessions=None):
    """AI ile sohbet - Hem proje hem veri"""
    global chatbot_conversation_history
    
    if not AI_AVAILABLE:
        return {"ok": False, "error": "AI kullanılamıyor. ANTHROPIC_API_KEY ayarlanmamış."}
    
    try:
        # Veri analizi gerekli mi kontrol et
        data_keywords = ['oturum', 'dikkat', 'uykululuk', 'performans', 'son', 'karşılaştır', 
                         'en iyi', 'en kötü', 'ortalama', 'risk', 'trend', 'kaç', 'veri']
        needs_data = any(keyword in user_query.lower() for keyword in data_keywords)
        
        # Context oluştur
        context = ""
        if needs_data:
            sessions_list = get_sessions_data()
            context = analyze_session_data(sessions_list, user_query)
        else:
            context = PROJECT_GUIDE
        
        # Sistem mesajı
        system_message = """Sen "Akıllı Sürücü İzleme Sistemi" projesinin AI asistanısın.

GÖREVLER:
1. Proje özelliklerini açıkla (nasıl kullanılır, ne yapar)
2. Kullanıcının oturum verilerini analiz et
3. Karşılaştırmalar yap
4. Trendleri göster
5. Öneriler sun

VERİ ANALİZİ:
- "X. oturum" derse, listedeki [X] numaralı oturumu kullan
- "Son 5 oturum" derse, ilk 5 oturumu analiz et
- "Performansım nasıl?" derse, genel istatistikleri göster
- "Karşılaştır" derse, metrikleri yan yana koy

PROJE REHBERİ:
- "Nasıl başlatırım?" derse, adım adım anlat
- "PDF nasıl?" derse, PDF oluşturma yolunu göster
- "Telegram?" derse, kurulum adımlarını ver

TARZ:
- Net ve anlaşılır
- Rakamlar göster
- Emoji kullan (az)
- Türkçe yaz
- Örnekler ver"""
        
        # Mesaj oluştur
        full_message = f"{context}\n\nKullanıcı: {user_query}"
        
        # Sohbet geçmişi
        chatbot_conversation_history.append({
            "role": "user",
            "content": full_message if len(chatbot_conversation_history) == 0 else user_query
        })
        
        # API çağrısı
        response = anthropic_client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=1200,
            system=system_message,
            messages=chatbot_conversation_history[-10:]
        )
        
        assistant_reply = response.content[0].text
        
        chatbot_conversation_history.append({
            "role": "assistant",
            "content": assistant_reply
        })
        
        return {"ok": True, "response": assistant_reply}
        
    except Exception as e:
        import traceback
        error_detail = traceback.format_exc()
        print(f"[HATA] chat_with_ai: {error_detail}")
        return {"ok": False, "error": f"AI hatası: {str(e)}"}

def clear_chat_history():
    """Sohbet geçmişini temizle"""
    global chatbot_conversation_history
    chatbot_conversation_history = []
    return {"ok": True}

def setup_ai_routes(app, list_sessions_func):
    """AI Chat route'ları - DÜZELTİLDİ"""
    global _list_sessions_func
    _list_sessions_func = list_sessions_func  # ← Fonksiyonu kaydet
    
    print(f"[OK] list_sessions fonksiyonu ayarlandı: {callable(_list_sessions_func)}")
    
    @app.route("/api/ai_chat", methods=["POST"])
    def ai_chat_endpoint():
        data = request.get_json() or {}
        query = data.get("query", "").strip()
        query_type = data.get("type", "general")
        sessions = data.get("sessions", [])
        
        if not query:
            return jsonify({"ok": False, "error": "Soru boş"})
        
        result = chat_with_ai(query, query_type, sessions)
        return jsonify(result)
    
    @app.route("/api/ai_chat/clear", methods=["POST"])
    def ai_chat_clear():
        result = clear_chat_history()
        return jsonify(result)

In [ ]:
print("\n" + "=" * 60)
print("🚗 Akıllı Sürücü İzleme Sistemi")
print("=" * 60)
print("📡 Server: http://localhost:5000")
print("=" * 60 + "\n")

res = test_telegram_connection()
if res.get("ok"):
    print(f"✅ Telegram Bot: @{res.get('bot_username')}")
else:
    print(f"❌ Telegram: {res.get('error')}")

# AI Chatbot'u başlat
init_chatbot(sessions_dir=SESSIONS_DIR)

# Route'ları ekle  
setup_ai_routes(app, list_sessions)

print("=" * 60 + "\n")

app.run(host="0.0.0.0", port=5000, debug=False)



🚗 Akıllı Sürücü İzleme Sistemi
📡 Server: http://localhost:5000

✅ Telegram Bot: @akilli_surucu_bot
[OK] AI Chatbot (Proje Rehberi + Veri Analizi) başlatıldı ✓
[OK] list_sessions fonksiyonu ayarlandı: True

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.101:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Dec/2025 12:12:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Dec/2025 12:12:42] "GET /api/telegram/test HTTP/1.1" 200 -
127.0.0.1 - - [24/Dec/2025 12:12:53] "POST /start_drive HTTP/1.1" 200 -
127.0.0.1 - - [24/Dec/2025 12:12:53] "GET /api/telegram/test HTTP/1.1" 200 -


Tam Surucu Izleme Sistemi
[OK] Duygu modeli yuklendi
[OK] Goz yonu tespit sistemi hazir
[INFO] YOLOv5 modeli yukleniyor...


Using cache found in C:\Users\gts_-/.cache\torch\hub\ultralytics_yolov5_master


[UYARI] YOLOv5 yuklenemedi: No module named 'ultralytics'
[INFO] Sadece el-tabanlı tespit kullanilacak
[OK] MediaPipe Hands yuklendi
[OK] Telefon tespit sistemi hazir
[OK] Ses sistemi baslatildi
[OK] Veri kayit sistemi hazir
[OK] Ana sürücü profili yüklendi: driver_profile.npy
KONTROLLER:
  q: Cikis ve oturumu kaydet
  p: Ana surucu olarak kaydet/guncelle (4.0 sn kayit)
[OK] Sistem baslatildi, kamera hazir

[INFO] Cikis yapiliyor...

[INFO] Oturum kaydediliyor...

[LOG] Oturum kaydedildi: sessions\surucu_oturumu_20251224_121256.json
[LOG] Süre: 23 saniye
[OK] Sistem kapatildi
